In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt

In [16]:
df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_data.csv')

In [19]:
X_train = df_train.iloc[:, :-1].values  # All columns except the last as features
y_train = df_train.iloc[:, -1].values   # Last column as target
X_test = df_test.iloc[:, 1:].values   # All columns except the first as features

In [20]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [7]:
# Define the FCNN model
model = Sequential([
    Dense(128, activation='relu', input_dim=X_train.shape[1]),  # Input layer
    Dropout(0.3),  # Regularization to avoid overfitting
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(1, activation='linear')  # Output layer for regression
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               5248      
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                 65        
                                                                 
Total params: 13569 (53.00 KB)
Trainable params: 13569 (53.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [48]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,  # Use 20% of training data for validation
    epochs=100,  # Number of epochs (adjust as needed)
    batch_size=32,  # Batch size
    verbose=1  # Display training progress
)

Epoch 1/100
3236/3236 [==============================] - 5s 1ms/step - loss: 0.4629 - mae: 0.5061 - val_loss: 0.4062 - val_mae: 0.4981
Epoch 2/100
3236/3236 [==============================] - 4s 1ms/step - loss: 0.4554 - mae: 0.5040 - val_loss: 0.5354 - val_mae: 0.5769
Epoch 3/100
3236/3236 [==============================] - 4s 1ms/step - loss: 0.4621 - mae: 0.5057 - val_loss: 0.4578 - val_mae: 0.5329
Epoch 4/100
3236/3236 [==============================] - 4s 1ms/step - loss: 0.4611 - mae: 0.5058 - val_loss: 0.4284 - val_mae: 0.5103
Epoch 5/100
3236/3236 [==============================] - 4s 1ms/step - loss: 0.4594 - mae: 0.5045 - val_loss: 0.4020 - val_mae: 0.4949
Epoch 6/100
3236/3236 [==============================] - 4s 1ms/step - loss: 0.4601 - mae: 0.5047 - val_loss: 0.4321 - val_mae: 0.5151
Epoch 7/100
3236/3236 [==============================] - 5s 1ms/step - loss: 0.4582 - mae: 0.5032 - val_loss: 0.4854 - val_mae: 0.5476
Epoch 8/100
3236/3236 [==============================] 

In [49]:
y_pred = model.predict(X_test)

2016/2016 [==============================] - 2s 747us/step


In [50]:
predictions_df = pd.DataFrame(y_pred)

In [51]:
column_from_df1 = df_test["Unnamed: 0"]
column_from_df2 = predictions_df.iloc[:, -1]

In [52]:
predictions_df = pd.concat([column_from_df1, column_from_df2], axis=1)

In [53]:
predictions_df.columns = ['id', 'target_feature']

In [54]:
predictions_df

,id,target_feature
0,2016,8.252352
1,2017,8.520462
2,2018,8.795734
3,2019,8.135920
4,2020,7.254939
...,...,...
64507,193531,5.753579
64508,193532,5.429995
64509,193533,5.458993
64510,193534,5.676717


In [55]:
print(predictions_df.dtypes)

id                  int64
target_feature    float32
dtype: object


In [56]:
sample_submission = pd.read_csv("sample_submission.csv")

In [57]:
sample_submission

,id,target_feature
0,2016,5.0
1,2017,5.0
2,2018,5.0
3,2019,5.0
4,2020,5.0
...,...,...
64507,193531,5.0
64508,193532,5.0
64509,193533,5.0
64510,193534,5.0


In [58]:
print(sample_submission.dtypes)

id                  int64
target_feature    float64
dtype: object


In [59]:
# Save submission as CSV file
predictions_df.to_csv('D:\Python\Predict the wind speed at a wind turbine\submissions\\oleg_bissing_submission_2.csv', index=False)